# Build my own deep learning functions

## Purpose

Learn by practicing. Notations follow Andrew Ng's Coursera deep learning course.

### Build the forward_prop function
Note: If there is error while importing python modules while running this notebook in vscode, make sure the both the vscode python interpreter and ipython kernal are both set properly. 

### Model structure
The model consist of L-1 relu layers and one sigmoid layer.

In [18]:
import numpy as np
import h5py

import h5py


In [ ]:
# Below function taken from course assignments
def load_data():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:])
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [11]:
def relu(Z):
    '''
    Z: Input to the activate function

    A: Output of the relu function
    '''
    A = np.maximum(0, Z)
    assert(np.min(A) >= 0.0)
    return A


In [2]:
def forward_prop(X, Y, parameters):
    '''
    # TODO: What's the row # variable?
    X: Input data. (n0???, m). n0: feature #; m: # of examples
    Y: Labels. (1, m)
    parameters: The model parameters

    J: the cost
    '''
    L = len(parameters) // 2
    Xl = X
    for l in range(1, L):
        # print('l=', l)
        W = parameters['W'+str(l)]
        b = parameters['b'+str(l)]

        assert(W.shape[0] == b.shape[0])
        Z = np.dot(W, Xl) + b
        A = relu(Z)
        Xl = A
    ZL = np.dot(parameters['W'+str(L)], Xl) + parameters['b'+str(L)]
    AL = 1 / (1 + np.exp(-ZL)) # sigmoid

    J = - Y * np.log(AL) - (1 - Y) * np.log(1 - AL)

    return J


In [3]:
layer_dims = [4, 4, 1]
parameters = {}
parameters['W1'] = 1.1
parameters['b1'] = 1.2
parameters['W2'] = 2.1
parameters['b2'] = 2.2
parameters['W3'] = 3.1
parameters['b3'] = 3.2
forward_prop(0, 0, parameters)


AttributeError: 'float' object has no attribute 'shape'